# Advent of Code 2020 - Day 8

In [82]:
enum class Operation(val id: String) {
    ACCUMULATOR("acc"),
    JUMP("jmp"),
    NO_OPERATION("nop"),
    ;

    companion object {
        private val idMap: Map<String, Operation> = Operation.values().associateBy(Operation::id)
        fun fromId(id: String): Operation = idMap.getValue(id)
    }
}

data class Instruction(val operation: Operation, val argument: Int)

In [83]:
import java.io.File

val instructions: List<Instruction> = File("Day8.input.txt")
    .bufferedReader()
    .readLines()
    .map { line -> line.split(" ") }
    .map { (op, arg) -> Instruction(operation = Operation.fromId(op), argument = arg.toInt()) }

## Part 1

Execute the `instructions` with the following `operation` rules:
- `ACCUMULATOR` increases or decreases a global accumulator value by the given `argument`. The instruction immediately after it is then executed.
- `JUMP` will "jump" to a new instruction relative to its current position where the `argument` is an offset.
- `NO_OPERATION` does nothing and the instruction immediately after it is run.

Find the value in the accumulator before a second instruction is executed a second time.

In [84]:
fun Int.execute(instruction: Instruction, accumulator: Int): Pair<Int, Int> =
    when (instruction.operation) {
        Operation.ACCUMULATOR -> (this + 1) to (accumulator + instruction.argument)
        Operation.JUMP -> (this + instruction.argument) to accumulator
        Operation.NO_OPERATION -> (this + 1) to accumulator
    }

fun List<Instruction>.execute(): Int {
    var index = 0
    var accumulator = 0
    val instructionsExecuted: MutableMap<Int, Instruction> = mutableMapOf()

    while(!instructionsExecuted.containsKey(index)) {
        instructionsExecuted[index] = instructions[index]
        index.execute(instructions[index], accumulator).let {
            index = it.first
            accumulator = it.second
        }
    }

    return accumulator
}

instructions.execute()

1766

### Notes

Another follow the steps problem. I think writing this part as perhaps a `tailrec fun` could have been interesting. I don't really like that `Int::execute` returns a `Pair<Int, Int>` and I think the code for `List<Instructions>::execute` could be a bit better as well.

## Part 2

Fix the program so that it terminates without an infinite loop by changing exactly one `JUMP` to `NO_OPERATION` or vice versa. Find the value in the accumulator after it terminates.

In [85]:
data class Console(
    val index: Int = 0,
    val accumulator: Int = 0,
    val executed: Map<Int, Instruction> = mapOf(),
)

fun Console.execute(instruction: Instruction): Console? =
    if (executed.containsKey(index)) null
    else index.execute(instruction, accumulator).let { (newIndex, newAccumulator) ->
        Console(index = newIndex, accumulator = newAccumulator, executed = executed + (index to instruction))
    }

tailrec fun List<Instruction>.exec(console: Console = Console()): Console =
    console.execute(this[console.index])
        .let { if (it != null) exec(it) else console }

println("Bonus Part 1 - ${instructions.exec().accumulator}")

tailrec fun List<Instruction>.execOrNull(console: Console = Console()): Console? =
    if (console.index < size) console.execute(this[console.index]) .let { if (it != null) execOrNull(it) else it }
    else console

fun fix(instruction: Instruction): Instruction =
    when (instruction.operation) {
        Operation.JUMP -> instruction.copy(operation = Operation.NO_OPERATION)
        Operation.NO_OPERATION -> instruction.copy(operation = Operation.JUMP)
        else -> instruction
    }

fun List<Instruction>.terminate(): Console? {
    instructions.forEachIndexed { index, _ ->
            instructions.toMutableList().apply { this[index] = fix(this[index]) }
                .execOrNull()
                .let { if (it != null) return it }
    }
    return null
}

instructions.terminate()?.accumulator

Bonus Part 1 - 1766


1639

### Notes

Ended up with a better implementation for part one I think. Spent a bit of time wondering why I had an `IndexOutOfBoundsException` and then realized that my `execOrNull` function hand't accounted for the scenario where it actually worked, since previously the problem statement relied on the fact it would never finish. Should definitely keep this in mind.

I'm also not quite happy with the implementation for `List<Instruction>::terminate`. Couldn't find a nice way to `firstNotNull` with index and a way to easily replace an element in a `List`. Both should probably be a generic utility extension?

The idea here is to break down the problem by just working against a single "state" of the `Console`. 